# Integrantes

Mauricio Lemus - 22461

Hugo Rivas - 22500

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import itertools
import pandas as pd
import numpy as np
import random

# Fijar semilla para reproducibilidad
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [5]:
# Transformaciones: a tensor y normalización (media=0.1307, std=0.3081 en MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Descarga y carga
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


100.0%
100.0%
100.0%
100.0%


In [6]:
class MLP(nn.Module):
    def __init__(self, layer_sizes, activation_fn):
        super().__init__()
        layers = []
        for in_dim, out_dim in zip(layer_sizes[:-1], layer_sizes[1:]):
            layers.append(nn.Linear(in_dim, out_dim))
            # no añadimos activación después de la última capa
            if out_dim != layer_sizes[-1]:
                layers.append(activation_fn())
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # aplanar 28×28 → 784
        return self.net(x)


In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    return 100. * correct / len(test_loader.dataset)


In [8]:
configs = {
    'A': {'layers':[784,128,10],    'act':nn.ReLU,    'lr':0.01,  'batch':64,  'epochs':10},
    'B': {'layers':[784,256,128,10],'act':nn.Sigmoid, 'lr':0.005, 'batch':128, 'epochs':15},
    'C': {'layers':[784,512,256,128,10],'act':nn.ReLU,'lr':0.001,'batch':32, 'epochs':20}
}
results = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for name, cfg in configs.items():
    # DataLoaders según batch_size
    train_loader = DataLoader(train_dataset, batch_size=cfg['batch'], shuffle=True)
    test_loader  = DataLoader(test_dataset,  batch_size=cfg['batch'], shuffle=False)

    # Modelo, optimizador
    model = MLP(cfg['layers'], cfg['act']).to(device)
    optimizer = optim.SGD(model.parameters(), lr=cfg['lr'])

    # Entrenamiento
    for ep in range(1, cfg['epochs']+1):
        train(model, device, train_loader, optimizer, ep)

    # Evaluación
    acc = test(model, device, test_loader)
    results[name] = acc
    print(f"Configuración {name}: Test Accuracy = {acc:.2f}%")


Configuración A: Test Accuracy = 96.27%
Configuración B: Test Accuracy = 76.20%
Configuración C: Test Accuracy = 94.51%


In [16]:
from itertools import product

grid = {
    'lr': [0.0001,0.001, 0.005, 0.01,0.1],
    'batch': [32, 64, 128,]
}
best = {'acc': 0}
for lr, batch in product(grid['lr'], grid['batch']):
    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    test_loader  = DataLoader(test_dataset,  batch_size=batch, shuffle=False)

    # Modelo
    model = MLP(configs['A']['layers'], configs['A']['act']).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)

    # Entrenamiento breve (p. ej. 5 épocas)
    for ep in range(1, 11):
        train(model, device, train_loader, optimizer, ep)

    acc = test(model, device, test_loader)
    print(f"lr={lr}, batch={batch} → acc={acc:.2f}%")
    if acc > best['acc']:
        best = {'lr':lr, 'batch':batch, 'acc':acc}

print("Mejor combinación:", best)


lr=0.0001, batch=32 → acc=85.79%
lr=0.0001, batch=64 → acc=81.35%
lr=0.0001, batch=128 → acc=71.57%
lr=0.001, batch=32 → acc=92.50%
lr=0.001, batch=64 → acc=91.16%
lr=0.001, batch=128 → acc=89.64%
lr=0.005, batch=32 → acc=96.21%
lr=0.005, batch=64 → acc=94.71%
lr=0.005, batch=128 → acc=93.27%
lr=0.01, batch=32 → acc=97.19%
lr=0.01, batch=64 → acc=96.17%
lr=0.01, batch=128 → acc=94.98%
lr=0.1, batch=32 → acc=98.05%
lr=0.1, batch=64 → acc=97.19%
lr=0.1, batch=128 → acc=98.03%
Mejor combinación: {'lr': 0.1, 'batch': 32, 'acc': 98.05}


In [17]:
# Agregar la mejor variante del grid search como "A_tuned"
results['A_tuned'] = best['acc']
df = pd.DataFrame([
    {'Red': k, 'Accuracy (%)': v} for k, v in results.items()
]).sort_values('Accuracy (%)', ascending=False).reset_index(drop=True)

print(df)


       Red  Accuracy (%)
0  A_tuned         98.05
1        A         96.27
2        C         94.51
3        B         76.20


### Evaluación y Conclusiones


**¿Qué hiperparámetros influyeron más en el rendimiento del modelo?**

Los hiperparámetros que más influyeron en la mejora del rendimiento fueron:

- **Learning rate**: Tuvo un impacto significativo, ya que un valor muy alto llevó a inestabilidad y uno muy bajo ralentizó el aprendizaje. Encontrar un valor intermedio como `0.005` o `0.001` permitió un entrenamiento más efectivo.
- **Número de capas ocultas y neuronas**: Las arquitecturas más profundas (como la red C) mostraron mejor capacidad de generalización, aunque con más riesgo de sobreajuste.
- **Batch size**: Influyó en la estabilidad y rapidez del entrenamiento. Tamaños pequeños como 32 proporcionaron mayor precisión en el gradiente, mientras que tamaños medianos como 64 ofrecieron un buen balance entre rendimiento y velocidad.
- **Función de activación**: ReLU mostró mejor rendimiento general comparado con Sigmoid, debido a su comportamiento no saturante y mejor propagación del gradiente.

El ajuste de hiperparámetros como la tasa de aprendizaje y la arquitectura de la red fue clave para mejorar el rendimiento. El grid search permitió encontrar una configuración óptima para la red A.
